In [2]:
import jupyter
from tweet_browser_test import tweet_browser as tb
import ipywidgets as widgets
import voila
from matplotlib import pyplot as plt
from ipysheet import sheet, cell, row, column, cell_range
from IPython.display import display
import pandas as pd
import io


###### Settings #########
fileName = "allCensus_sample.csv"
colHeaders = ["SenderScreenName", "CreatedTime", "Message"]   
tweetsPerPage = 25 # Must be smaller than dataset size


s = tb.createSession(fileName)
out = widgets.Output()

def resetDisplay(b = None):
    global stack, sampleSelector
    sampleSelector.max = s.currentSet.size
    out.clear_output(True)
    with out:
        display(randomSampleButton) 
    
def randomSample(b):
    if(s.currentSet.size <= tweetsPerPage):
        s.back()
    s.simpleRandomSample(tweetsPerPage)
    getTweets(b)
    
def searchKeyword(b):
    if searchType.value == 'Contains Any':
        s.searchKeyword(searchBox.value.split(' '), True)
    elif searchType.value == 'Contains All':
        s.searchKeyword(searchBox.value.split(' '))
    elif searchType.value == 'Advanced':
        s.advancedSearch(searchBox.value)
    elif searchType.value == 'Regular Expression':
        s.regexSearch(searchBox.value)
    else:
        return
    getTweets(b)
    
    
def getTweets(b):
    resetDisplay()
    htmlText = "<style>table, th, td {border: 1px solid black;} th, td {padding: 10px;}</style>"
    htmlText += "<table> <tr> "
    for header in colHeaders:
        htmlText += " <th> " + header + " </th> "
    dataSet = s.getCurrentSubset()
    for i in range(len(dataSet)):
        htmlText += " <tr> "
        for j in colHeaders:
            #print(type(dataSet.iloc[i].loc[j]))
            htmlText += " <td>" + str(dataSet.iloc[i].loc[j]) + "</td> "
        htmlText += " </tr> "
    table = widgets.HTML(value=htmlText + " </table>")
    with out:
        #print("Tweets displayed:", s.currentSet.size)
        display(table)

####### UI Widgets #######

# Random Sample
randomSampleButton = widgets.Button(description='Get Tweets', tooltip='Get Tweets')   
sampleSelector = widgets.IntSlider(value=300, min=0, max=s.currentSet.size, step=1, 
                description='Sample size', continuous_update=False, orientation='horizontal', readout=True, 
                readout_format='d', tooltip= 'The number of tweets that will be selected in the sample')
randomSampleBar = widgets.HBox([sampleSelector, randomSampleButton])
randomSampleButton.on_click(randomSample)

#Search Bar
searchBox = widgets.Text(value='', placeholder='Keyword', description='Input:')
searchButton = widgets.Button(description='Search', tooltip='Run search on current input')
ContainsAnyToolTip = 'Returns tweets with any typed keyword (seperated by spaces)'
ContainsAllToolTip = 'Returns tweets containing all typed space-separated keywords'
AdvancedSearchToolTip = 'Search for tweets that match the inputted logical expression'
RegularExpressionToolTip = 'Returns tweets that match regular expression'
searchType = widgets.ToggleButtons(options=['Contains Any', 'Contains All', 'Advanced', 'Regular Expression'],
    tooltips=[ContainsAnyToolTip, ContainsAllToolTip, AdvancedSearchToolTip, RegularExpressionToolTip])
searchBar = widgets.VBox([widgets.HBox([searchBox, searchButton]), searchType])
searchButton.on_click(searchKeyword)
    
with out:
    display(randomSampleButton)
    #display(stack)
    #print("Select a file to analyze:")
    #display(fileUp)
    
out

Output()